## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input

from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''

'''Resnet 50 架構'''
model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
model.summary()


Using TensorFlow backend.


C:\Users\Bill\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 14s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________

## 添加層數

In [2]:
x = model.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.1)(x)
predictions = Dense(units=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model深度： 179


## 鎖定特定幾層不要更新權重

In [3]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


C:\Users\Bill\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [5]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
50000/50000 [==============================] - 106s 2ms/step - loss: 1.2874 - acc: 0.5752
Epoch 2/100
50000/50000 [==============================] - 96s 2ms/step - loss: 0.9882 - acc: 0.6593
Epoch 3/100
50000/50000 [==============================] - 97s 2ms/step - loss: 0.8849 - acc: 0.6995
Epoch 4/100
50000/50000 [==============================] - 99s 2ms/step - loss: 0.8259 - acc: 0.7170
Epoch 5/100
50000/50000 [==============================] - 108s 2ms/step - loss: 0.7541 - acc: 0.7383
Epoch 6/100
50000/50000 [==============================] - 97s 2ms/step - loss: 0.7147 - acc: 0.7508
Epoch 7/100
50000/50000 [==============================] - 101s 2ms/step - loss: 0.6181 - acc: 0.7834
Epoch 8/100
50000/50000 [==============================] - 97s 2ms/step - loss: 0.5533 - acc: 0.8107
Epoch 9/100
50000/50000 [==============================] - 97s 2ms/step - loss: 0.5381 - acc: 

50000/50000 [==============================] - 97s 2ms/step - loss: 0.1063 - acc: 0.9679: 0s - loss: 0.1061 - acc: 0.
Epoch 75/100
50000/50000 [==============================] - 97s 2ms/step - loss: 0.1030 - acc: 0.9683
Epoch 76/100
50000/50000 [==============================] - 96s 2ms/step - loss: 0.0847 - acc: 0.9735
Epoch 77/100
50000/50000 [==============================] - 98s 2ms/step - loss: 0.0849 - acc: 0.9741
Epoch 78/100
50000/50000 [==============================] - 97s 2ms/step - loss: 0.0827 - acc: 0.9746
Epoch 79/100
50000/50000 [==============================] - 97s 2ms/step - loss: 0.0845 - acc: 0.9739
Epoch 80/100
50000/50000 [==============================] - 98s 2ms/step - loss: 0.0771 - acc: 0.9764
Epoch 81/100
50000/50000 [==============================] - 97s 2ms/step - loss: 0.0773 - acc: 0.9765
Epoch 82/100
50000/50000 [==============================] - 96s 2ms/step - loss: 0.0725 - acc: 0.9778
Epoch 83/100
50000/50000 [==============================] - 97s 2m